In [35]:
import tensorflow as tf
import numpy as np
import base64
import keras
import os
import sys
from PIL import Image
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import csv

In [36]:
# Compress images

def resize(folder, fileName):
    filePath = os.path.join(folder, fileName)
    im = Image.open(filePath)
    w, h  = im.size
    newIm = im.resize((64, 64))
    newIm.save(filePath)

def bulkResize(imageFolder):
    imgExts = ["png", "jpg"]
    for path, dirs, files in os.walk(imageFolder):
        for fileName in files:
            ext = fileName[-3:].lower()
            if ext not in imgExts:
                continue
            resize(path, fileName)

imageFolder="/Users/Chrsitine/Desktop/Conjurers/images/" 
bulkResize(imageFolder)
print("Done resizing")

Done resizing


In [38]:
# Load training and testing data 

x_train = []
y_train = []
x_test = []
y_test = []

imgExts = ["png", "jpg"]
counter = -1
for path, dirs, files in os.walk("/Users/Chrsitine/Desktop/Conjurers/images/"):
    for fileName in files:
        ext = fileName[-3:].lower()
        if ext not in imgExts:
            continue
        else:
            filePath = os.path.join(path, fileName)
            img = Image.open(filePath)
            img_array = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
            x_train.append(img_array)
            label = np.zeros(10)
            label[counter] = 1
            # y_train.append(label)
            y_train.append(counter)
    counter += 1

imgExts = ["png", "jpg"]
counter = -1
for path, dirs, files in os.walk("/Users/Chrsitine/Desktop/Conjurers/test_images/"):
    for fileName in files:
        ext = fileName[-3:].lower()
        if ext not in imgExts:
            continue
        else:
            filePath = os.path.join(path, fileName)
            img = Image.open(filePath)
            img_array = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0], 3)
            x_test.append(img_array)
            label = np.zeros(10)
            label[counter] = 1
            # y_test.append(label)
            y_test.append(counter)
    counter += 1

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: " ,y_test.shape)

x_train.shape:  (974, 64, 64, 3)
y_train.shape:  (974,)
x_test.shape:  (70, 64, 64, 3)
y_test.shape:  (70,)


In [39]:
# Train model

batch_size = 56
num_classes = 10
epochs = 40

# input image dimensions
img_rows, img_cols = 64, 64

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Save trained model
model.save('trained_model.hdf5')
model.save_weights('trained_weights.hdf5')
print("Saved trained model")

score = model.evaluate(x_test, y_test, verbose=1)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

x_train shape: (974, 64, 64, 3)
y_train shape: (974,)
974 train samples
70 test samples
Train on 974 samples, validate on 70 samples
Epoch 1/40
974/974 [==============================] - 26s - loss: 2.4869 - acc: 0.1355 - val_loss: 2.1621 - val_acc: 0.1000
Epoch 2/40
974/974 [==============================] - 26s - loss: 2.0808 - acc: 0.2341 - val_loss: 2.0221 - val_acc: 0.2143
Epoch 3/40
974/974 [==============================] - 24s - loss: 1.8958 - acc: 0.3183 - val_loss: 1.7298 - val_acc: 0.3571
Epoch 4/40
974/974 [==============================] - 28s - loss: 1.6650 - acc: 0.4055 - val_loss: 1.4493 - val_acc: 0.5429
Epoch 5/40
974/974 [==============================] - 19s - loss: 1.3546 - acc: 0.5329 - val_loss: 1.1444 - val_acc: 0.6857
Epoch 6/40
974/974 [==============================] - 15s - loss: 1.0634 - acc: 0.6417 - val_loss: 1.0746 - val_acc: 0.6286
Epoch 7/40
974/974 [==============================] - 27s - loss: 0.8536 - acc: 0.7331 - val_loss: 0.8379 - val_acc: 0.7429

In [50]:
predict = model.predict(x_test, verbose=0)
count = 0
print("Pred Actual")
for i in range(predict.shape[0]):
    if np.argmax(predict[i]) != np.argmax(y_test[i]):
        print(np.argmax(predict[i]), "   ", np.argmax(y_test[i]))
        count += 1
print("Wrong count: ", count)

Pred Actual
3     0
5     0
3     0
2     3
0     3
4     5
3     8
Wrong count:  7


In [ ]:
# 0-indexing with one hot encoding is used for class labels:
idx_to_label = {0: "Coke zero", 1: "Oreo thins (original)", 2: "Cadbury chocolate fingers",
               3: "Diet coke", 4: "Hershey's drops", 5: "m&ms", 6: "Sun chips", 
               7: "Sour cream and onion Pringles", 8: "Coke (original)", 9: "Golden Oreo thins"}